In [1]:
sc

In [2]:
sqlCtx

In [10]:
data =sqlCtx.read.format("csv").option("header", True) \
.option("inferSchema",True) \
.option("delimiter", "\t") \
.load("file:/home/cloudera/amazon_reviews_us_Watches_v1_00.tsv.gz")

In [14]:
data.limit(2).toPandas()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31


In [21]:
data.select("star_rating").distinct().show()

+-----------+
|star_rating|
+-----------+
|          1|
|          2|
|          3|
|          4|
|          5|
+-----------+



In [22]:
data.registerTempTable("watches")

In [59]:
data1 = sqlCtx.sql("select star_rating, review_body from watches where star_rating == 1 group by star_rating, review_body")

In [41]:
data1.count()

1000

In [32]:

import re # needed to remove special character
from pyspark import Row

import json
from pyspark.sql import SQLContext
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer, CountVectorizer
from pyspark.mllib.clustering import LDA
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, LongType
from pyspark.sql.functions import *
from nltk.stem.porter import *
from pyspark.sql.types import *

In [60]:
data1rdd = data1.rdd

In [61]:
pattern1 = re.compile('\W+|\W+$|[^\w\s]+|_')
pattern2 = re.compile(r'\W*\b\w{1,2}\b')

#pattern1 = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

rdd = data1rdd \
    .mapValues(lambda x: pattern1.sub(' ', x)) \
    .mapValues(lambda x: pattern2.sub(' ', x))

df = rdd.toDF(schema=['file', 'text'])

In [62]:
row_with_index = Row(*["id"] + df.columns)

def make_row(columns):
    def _make_row(row, uid):
        row_dict = row.asDict()
        return row_with_index(*[uid] + [row_dict.get(c) for c in columns])

    return _make_row

f = make_row(df.columns)

indexed = (df.rdd
           .zipWithUniqueId()
           .map(lambda x: f(*x))
           .toDF(StructType([StructField("id", LongType(), False)] + df.schema.fields)))

In [63]:

tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
tokenized = tokenizer.transform(indexed)
print 'done'

# remove stop words
stopwordList = ['','get','got','also','really','me','my','myself','we',
 'our','ours','ourselves','you','your','yours','yourself','yourselves','he','him',
 'his','himself','she','her','hers','herself','it','its','itself','they',
 'them','their', 'theirs','themselves','what','which','who','whom','this',
 'that','these','those','am','is','are','was','were','be','been',
 'being','have','has','had','having','do','does','did','doing','a','an','the',
 'and','but','if','or','because','as','until','while','of','at','by','for',
 'with','about','against','between','into','through','during','before','after',
 'above','below','to','from','up','down','in','out','on','off','over','under','again',
 'further','then','once','here','there','when','where','why','how','all',
 'any','both','each','few','more','most','other','some','such','no','nor','not','only',
 'own','same','so','than','too','very','s','t','can','will''just','don',
 'should','now','d','ll','m','o','re','ve','y','ain', 'aren', 'couldn','didn',
 'doesn','hadn','hasn','haven','isn','ma','mightn','mustn','needn',
 'shan','shouldn','wasn','weren','won','wouldn']


# Removing Stop words
remover=StopWordsRemover(inputCol="tokens", outputCol="words" ,stopWords=stopwordList)
cleaned = remover.transform(tokenized)
print 'done'

#stem words
# Instantiate stemmer object
stemmer = PorterStemmer()

# Create stemmer python function
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        if len(t_stem) > 2:
            out_vec.append(t_stem)       
    return out_vec

# Create user defined function for stemming with return type Array<String>
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

# Create new df with vectors containing the stemmed tokens 
# Create new df with vectors containing the stemmed tokens 
vector_stemmed_df = (
    cleaned
        .withColumn("vector_stemmed", stemmer_udf("words"))
  )


# vectorize
cv = CountVectorizer(inputCol="vector_stemmed", outputCol="vectors")
print 'done'
count_vectorizer_model = cv.fit(vector_stemmed_df)
print 'done'
result = count_vectorizer_model.transform(vector_stemmed_df)

done
done
done
done


In [64]:
corpus = result.select(F.col('id').cast("long"), 'vectors').rdd \
    .map(lambda x: [x[0], x[1]])

In [65]:
# training data
lda_model = LDA.train(rdd=corpus, k=5, seed=12, maxIterations=10)
# extracting topics
topics = lda_model.describeTopics(maxTermsPerTopic=10)
# extraction vocabulary
vocabulary = count_vectorizer_model.vocabulary

In [58]:
for topic in range(len(topics)):
    print("topic {} : ".format(topic))
    words = topics[topic][0]
    scores = topics[topic][1]
    for word in range(len(words)):
        print(vocabulary[words[word]], "->", scores[word])

topic 0 : 
(u'watch', '->', 0.060781673589666)
(u'time', '->', 0.014287839356970554)
(u'work', '->', 0.013246304321076435)
(u'look', '->', 0.010901606695168048)
(u'one', '->', 0.01079050843574341)
(u'return', '->', 0.008826846018517201)
(u'band', '->', 0.008339117703619855)
(u'would', '->', 0.00829779803929313)
(u'like', '->', 0.007865306446118782)
(u'day', '->', 0.007723457313712524)
topic 1 : 
(u'watch', '->', 0.06314514555240586)
(u'time', '->', 0.014639522777564515)
(u'work', '->', 0.012716100846548133)
(u'look', '->', 0.010855964680423874)
(u'one', '->', 0.010710404326144512)
(u'return', '->', 0.008608957512747754)
(u'would', '->', 0.008523473508861984)
(u'band', '->', 0.008180929130094142)
(u'like', '->', 0.007644169254708581)
(u'day', '->', 0.0074096512300357385)
topic 2 : 
(u'watch', '->', 0.06227913119291515)
(u'time', '->', 0.015073561580536146)
(u'work', '->', 0.01297282494378452)
(u'look', '->', 0.011227993586512261)
(u'one', '->', 0.010511664106660365)
(u'return', '->', 0.